In [1]:
import os.path as op
from glob import glob
# set path to SpectralEvents if necessary
#import sys
#sys.path.append('/home/ryan/SpectralEvents')

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns

import spectralevents_functions as se

sns.set()

In [2]:
# dataset parameters
data_dir = 'test_data'  # relative path to data directory
subj_ids = [str(id) for id in range(1, 10 + 1)]  # subject IDs 1-10

n_subjs = len(subj_ids)  # number of subjects
n_trials = 200           # number of trials per subject
n_samps = 600            # number of samples per trial
Fs = 600                 # sampling rate (Hz)

In [3]:
# load data
hit_trials = list()
meg_data = list()
for id_idx, id in enumerate(subj_ids):
    fname = op.join(data_dir, 'prestim_humandetection_600hzMEG_subject' + id + '.mat')
    raw_data = loadmat(fname)
    hit_trials.append(np.nonzero(raw_data['YorN'].squeeze()))  # indices of hit trials
    meg_data.append(raw_data['prestim_raw_yes_no'])  # MEG time series (trials x samples)

In [4]:
# investigate data structure
meg_data[0].shape  # trials x samples for Subj 1

(200, 600)

In [5]:
# time-frequency response (TFR) parameters
freqs = list(range(1, 60 + 1))   # TFR: fequency values (Hz) over which to calculate
morlet_width = 7  # TFR: width parameter of Morlet wavelet

# spectral event analysis parameters
eventBand = [15, 29]          # frequency range (Hz) of spectral events (beta band)
thrFOM = 6; # Factors of Median threshold (see Shin et al. eLife 2017 for details concerning this value)

In [12]:
# Run spectral event analysis per dataset (Dataset loop)
TFR = list()
specEvents = list()
for subj_idx, subj_data in enumerate(meg_data):

    # calculate time-frequency response (TFR)
    thisTFR = se.spectralevents_ts2tfr(subj_data, freqs, Fs, morlet_width)
    tVec = np.arange(n_samps) / Fs
    TFR.append(thisTFR)

    # Normalize the TFR data [tr x f x t] to the median value per frequency band
    numTrials, numFreqBins, numSamples = thisTFR.shape
    TFR_order = np.transpose(thisTFR, axes=[1, 0, 2])  # [f x tr x t]
    TFR_reshape = np.reshape(TFR_order, (numFreqBins, numTrials * numSamples))
    TFRmeds = np.median(TFR_reshape, axis=1)  # f vector
    TFRmeds_expanded = np.transpose(np.tile(TFRmeds, (numSamples, numTrials, 1)), axes=[1, 2, 0])
    thisTFR_norm = thisTFR / TFRmeds_expanded

    # Find local maxima in TFR
    thisSpecEvents = se.find_events(eventBand, thrFOM, tVec, freqs, thisTFR, Fs)
    #thisSpecEvents = pd.DataFrame( thisSpecEvents )
    specEvents.append(thisSpecEvents)



In [ ]:
    # Extract event attributes for this test data
    classes = np.unique(thisClassLabels)

    # Plot results for each class of trial
    for clss in classes:

        # Get TFR, time course, and trial IDs for this class of trials only
        trial_inds = np.nonzero(thisClassLabels == clss)[0]
        classTFR = thisTFR[trial_inds, :, :]
        classTFR_norm = thisTFR_norm[trial_inds, :, :]
        classData = thisData[trial_inds, :]

        # Get events data for this class only, and update trial indices to be consecutive
        #   starting at 0
        event_trials = [(event['Trial'] in trial_inds) for event in thisSpecEvents]
        classEvents = thisSpecEvents[event_trials]

        # Make figure
        fig, axs = se.plot_avg_spectrogram(classEvents, classData, classTFR,
                                           classTFR_norm, tVec, freqs,
                                           eventBand)
        # Add title
        axs[0, 0].set_title('DataSet ' + str(subj_idx) + ', Trial class ' + str(clss))